# cronuts

The Simplex Bakery is trying to determine how many dozens of cronuts to bake each day. Customers arrive at the bakery according to an exponential interarrival time distribution with a mean of 12 minutes. The dozens of cronuts that each customer orders follows a Binomial distribution with 4 trials and success probability 0.3.

Cronuts sell for \$20 per dozen. 
They cost \$12 per dozen to make. 
All cronuts not sold at the end of the day are sold at half price to a local stale baked good distributor.

Due to mixing size constraints, cronuts must be made in batches of 20 dozen, anywhere between 0 and 200 dozen. 

Assume that the Simplex Bakery is open for 6 hours a day.

Using 50 replications, how many dozen should the bakery bake per day to maximize its profit?

In [1]:
##### Setup #####
# Import various functions from NumPy
from numpy import mean

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, rand, binomial, exponential

# Import everything from SimPy
from SimPy.Simulation import *

In [2]:
##### Parameters #####
class P:
    # Revenue per dozen cronuts
    revenue = 20
    
    # Cost per dozen cronuts
    cost = 12
    
    # Number of dozen cronuts baked
    nCronuts = None
    
    # Interarrival time distribution: mean 12
    interarrivalTimeMean = 12
    
    # Cronuts per customer distribution: 10 trials, success probability 0.3
    cronuts_n = 4
    cronuts_p = 0.3
    
    # Bakery is open for 6 hours
    simulationTimeMax = 6 * 60

    
##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, determines how many cronuts she wants
        cronutsWanted = binomial(n = P.cronuts_n, p = P.cronuts_p)
        
        # Are there enough cronuts for this customer?
        if cronutsWanted > L.cronuts.amount:
            yield get, self, L.cronuts, L.cronuts.amount
        else:
            yield get, self, L.cronuts, cronutsWanted
        
        # Check if 
            
# Entrance
class Entrance(Process):
    def behavior(self):
        # Start customer for number of customers
        nCustomers = 0
        
        while True:
            # Interarrival time
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create customer using the template defined in the Customer class
            c = Customer(name = "Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())
            
            # Count this new customer
            nCustomers += 1


##### Levels #####
class L:
    cronuts = None
    
            
##### Simulation #####
def model(inputSeed):
    # Initialize SimPy
    initialize()
    
    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create level for cronuts
    L.cronuts = Level(initialBuffered = P.nCronuts)
    
    # Activate the entrance
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    # Compute profit
    profit = (P.revenue - P.cost) * (P.nCronuts - L.cronuts.amount) + (0.5 * P.revenue - P.cost) * L.cronuts.amount
    
    # Return performance measures
    return profit

In [3]:
# Number of replications
n = 50

# Run n replications for different initial inventory of cronuts
for P.nCronuts in [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]:
    profitObs = [model(inputSeed = 123*i) for i in range(n)]
    profitSM = mean(profitObs)
    print("Dozens baked = {0}, mean profit = {1}".format(P.nCronuts, profitSM))

Dozens baked = 0, mean profit = 0.0
Dozens baked = 20, mean profit = 159.8
Dozens baked = 40, mean profit = 268.0
Dozens baked = 60, mean profit = 246.2
Dozens baked = 80, mean profit = 206.2
Dozens baked = 100, mean profit = 166.2
Dozens baked = 120, mean profit = 126.2
Dozens baked = 140, mean profit = 86.2
Dozens baked = 160, mean profit = 46.2
Dozens baked = 180, mean profit = 6.2
Dozens baked = 200, mean profit = -33.8


Based on these simulations, the Simplex Bakery should bake 40 dozen cronuts per day.